In [3]:
# pip install fastparquet
# pip install pyarrow

import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
punc = '''=--*!()-[]{};:'"\,<>./?@#%^&*_~'''
import time
import IPython

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kjadbaba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_parquet("DSC180B.parquet")

In [4]:
len(df)

496101

In [23]:
df.head()

,transaction_date,amount,memo,new_category
8,2018-03-06,15.25,POS CASINO BAR @ SPOTL - MEMO=PURCHASE 03/02 C...,Entertainment
39,2018-09-29,16.04,BEST BUY GRAND REGENCY BRANDON F,General Merchandise
45,2018-10-17,10.36,CORNER STORE ARLINGTON TX 10/17 Purchase $5.3...,Automotive
52,2017-07-11,4.63,SPEEDWAY IN BEDFORD IN 07/10 DEBIT_CARD,Automotive
55,2018-09-10,223.00,PAYMENT FOR AMZ STORECARD WEB ID: ACH_DEBIT,General Merchandise


## Data Cleaning

In [26]:
def removeStop(sentence):
    """
    For each row in dataset, removes stopwords and lowercases every word
    """
    words = ""
    for j in punc:
        sentence = sentence.replace(j, "")
    for word in sentence.split():
        if word.lower() not in stopwords.words('english') and word.lower() not in punc:
            words = words + " " + word.lower()
#             words.append(word.lower())
    return words

df['Cleaned Text'] = df['memo'].apply(removeStop)

In [30]:
df.to_parquet("df_cleaned.parquet")

In [4]:
clean_df = pd.read_parquet("df_cleaned.parquet")

In [5]:
len(clean_df)

496101

In [1]:
import pandas as pd
pd.read_parquet("df_cleaned.parquet")

,transaction_date,amount,memo,new_category,Cleaned Text
8,2018-03-06,15.25,POS CASINO BAR @ SPOTL - MEMO=PURCHASE 03/02 C...,Entertainment,pos casino bar spotl memopurchase 0302 coache...
39,2018-09-29,16.04,BEST BUY GRAND REGENCY BRANDON F,General Merchandise,best buy grand regency brandon f
45,2018-10-17,10.36,CORNER STORE ARLINGTON TX 10/17 Purchase $5.3...,Automotive,corner store arlington tx 1017 purchase $536 ...
52,2017-07-11,4.63,SPEEDWAY IN BEDFORD IN 07/10 DEBIT_CARD,Automotive,speedway bedford 0710 debitcard
55,2018-09-10,223.00,PAYMENT FOR AMZ STORECARD WEB ID: ACH_DEBIT,General Merchandise,payment amz storecard web id achdebit
...,...,...,...,...,...
1999981,2021-02-19,24.24,DBT/WDR CASEYS GEN STORE FENNIMORE WI,Automotive,dbtwdr caseys gen store fennimore wi
1999982,2021-01-19,120.60,POS Debit - DDA KOHLS CLIVE IA #,General Merchandise,pos debit dda kohls clive ia
1999983,2022-02-25,6.47,STARBUCKS STORE MAPLE VALLEY WA 02/24,Food and Beverages,starbucks store maple valley wa 0224
1999991,2021-01-11,44.98,POS Debit - Visa Check Card - APPLE.COM/BILL ...,General Merchandise,pos debit visa check card applecombill ca


In [ ]:
# Other things to do for cleaning

# Make new features with: Amount and date

### Manual Categorization

Idea. If certain words directly correspond with a particular category 95% of the time or more we can automatically label that category. Ex: Shell == Automotive. ONLY RUN ONCE (TAKES A LONG TIME TO RUN). Goal is to get dictionary and save it then never run the code again.

In [10]:
# # Getting every single word in dataset

# big_list = np.sum(clean_df["Cleaned Text"]).split(" ")[1:]

# # Trying to get value_counts. Making df
# comb_df = pd.DataFrame(big_list, columns = ['words'])
# top_words_df = comb_df.value_counts().to_frame().reset_index()
# top_words = list(top_words_df["words"])
# # mini["Cleaned Text String"] = mini["Cleaned Text"].apply(lambda x:''.join(x))

# # Value Counts
# top_words_df = comb_df.value_counts().to_frame().reset_index()

# # Now into list of top x words

# top_words = list(top_words_df["words"])[:5000]

# def find_rows_containing_word(list_of_words):
#     """
#     Creating a dictionary of words that pass 95% threshold. Input every word in dataset, output dictionary with key 
#     as word and value as category
#     """
#     dictionary = {}
#     for word in list_of_words:
#         try:
#             output = clean_df.loc[clean_df["Cleaned Text"].str.contains(word)]
#             if len(output) != 0:
#                 val_counts = output.new_category.value_counts(normalize = True).to_frame()
#                 index = list(val_counts.index)[0]
#                 top_percentage = val_counts.iloc[0][0]
#                 if top_percentage >= .90:
#                     dictionary[word] = index + ": " + str(top_percentage)
#         except:
#             continue
#     return dictionary

# cheat_dict = find_rows_containing_word(top_words)

# cheat_df = pd.DataFrame.from_dict(cheat_dict, orient = "index")
# categories = []
# percents = []
# for j in range(len(cheat_df)):
#     lst = cheat_df.iloc[j][0].split(": ")
#     category = lst[0]
#     percent = round(float(lst[1]), 2)
#     categories.append(category)
#     percents.append(percent)
# words = list(cheat_df.index)
# cheat_df = pd.DataFrame(list(zip(words, categories, percents)), columns = ["word", "category", "percent"])
# cheat_df.to_csv("cheat_df.csv")